# Create Dataset
This notebook extracts protein secondary structure information from the Protein Data Bank for a set of representative protein chains and assigns a fold classification.

---
**Rule 2: Make a Habit of Documenting Your Process, Not Just Results.** Here we describe the steps how to produce the dataset.

**Rule 3: Document Your Entire Workflow so Your Notebook Can Become a Pipeline.** Besides documenting all steps, the entire process of dataset creation from the original data files in the /data directory is automated. There are no manual steps.

**Rule 7: Share Your Data and Explain How to Use It.** To enable reproducibility we provide a /data directory with two data files and a README file with the download location and date.

---

In [1]:
# parameters
value_col = "foldClass" # fold class to be predicted

In [2]:
import pandas as pd
import numpy as np     
import pdbutils 

### Read Representative Set of PDB Chains
Protein chains in the Protein Data Bank are redundant. For example, there are more than 300 structures of hemoglobin in the PDB. To avoid biases in our analysis, we use a representative set of protein chains, i.e., a set of protein chains with minimal sequence identity among its members. For this analysis we downloaded a non-redundant set from the [PISCES website](http://dunbrack.fccc.edu/PISCES.php) with a maximum of 25% sequence identity and an X-ray resolution of <= 3.0 &#197;. 

Wang G, Dunbrack RL Jr (2005) PISCES: recent improvements to a PDB sequence culling server, Nucleic Acids Res. 33, W94-8. [doi: 10.1093/nar/gki402](https://doi.org/10.1093/nar/gki402)

In [3]:
rep = pdbutils.read_pisces_representatives('./data/cullpdb_pc25_res3.0_R1.0_d180920_chains14051.gz')
print("Number of representative PDB chains:", rep.shape[0])
rep.head()

Number of representative PDB chains: 14051


,length,Exptl.,resolution,R-factor,FreeRvalue,pdbChainId
0,330,XRAY,2.2,0.16,0.29,12AS.A
1,366,XRAY,2.1,0.19,0.26,16VP.A
2,348,XRAY,2.6,0.22,0.34,1A0I.A
3,413,XRAY,1.7,0.19,0.22,1A12.C
4,108,XRAY,2.0,0.21,0.25,1A1X.A


### Read Protein Sequence and Secondary Structure Data
Protein secondary structure is frequently assigned with the DSSP method.

Kabsch W, Sander C (1983) Dictionary of protein secondary structure: Pattern recognition of hydrogen‐bonded and geometrical features. Biopolymers 22, 2577–637. [doi 0.1002/bip.360221211](https://doi.org/10.1002/bip.360221211)

DSSP defines [8 classes of secondary structure](https://en.wikipedia.org/wiki/Protein_secondary_structure):

| DSSP Class  | Code |
| :---        | ---: |
| 4-turn helix (&#945; helix), min. length 4 residues | H |
| Isolated &#946;-bridge (single pair &#946;-sheet hydrogen bond formation) | B |
| Extended strand in parallel or anti-parallel &#946;-sheet conformation, min length 2 residues | E |
| 3-turn helix (3<sub>10</sub> helix), min. length 3 residues | G |
| 5-turn helix (&#960; helix), min. length 5 residues | I |
| Hydrogen bonded turn (3, 4 or 5 turn) | T |
| Bend (the only non-hydrogen-bond based assignment) | S |
| Coil (residues which are not in any of the above conformations) | C |

The [RCSB Protein Data Bank](https://www.rcsb.org) provides protein sequence and DSSP secondary structure assignments. The method below reads a local copy of this file and returns the data as a Pandas dataframe. 

The secondary_structure string has the same length as the sequence string and represents the secondary structure assignment for each amino acid residue.

In [4]:
ss = pdbutils.read_secondary_structure('./data/ss_dis.txt.gz')
print("Number of PDB chains:", ss.shape[0])
ss.head()

Number of PDB chains: 402007


,pdbChainId,sequence,secondary_structure
0,101M.A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,CCCCHHHHHHHHHHHHHHGGGHHHHHHHHHHHHHHHCGGGGGGCTT...
1,102L.A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,CCHHHHHHHHHCCEEEEEECTTSCEEEETTEEEESSSCTTTHHHHH...
2,102M.A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,CCCCHHHHHHHHHHHHHHGGGHHHHHHHHHHHHHHHCGGGGGGCTT...
3,103L.A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,CCHHHHHHHHHCCEEEEEECTTSCEEEETTEECCCCCCCCCHHHHH...
4,103M.A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,CCCCHHHHHHHHHHHHHHGGGHHHHHHHHHHHHHHHCGGGGGGCTT...


### Find the Intersection between the two Datasets
Note that the representative set contains about 14,000 protein chains, whereas the entire PDB contains more than 400,000 chains.

By merging the representative set of protein chains with the secondary structure dataset we obtain the intersection between the two sets.

In [5]:
df = ss.merge(rep, left_on='pdbChainId', right_on='pdbChainId', how='inner')
print("Number of representative chains with secondary structure data:", df.shape[0])
df.head()

Number of representative chains with secondary structure data: 13791


,pdbChainId,sequence,secondary_structure,length,Exptl.,resolution,R-factor,FreeRvalue
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,330,XRAY,2.2,0.16,0.29
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,366,XRAY,2.1,0.19,0.26
2,1A0I.A,VNIKTNPFKAVSFVESAIKKALDNAGYLIAEIKYDGVRGNICVDNT...,CTTCCCCEEEEECCHHHHHHHHHHHSSEEEEECCCSEEEEEEEETT...,348,XRAY,2.6,0.22,0.34
3,1A12.C,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,CCCCCCCCCCCCCCCCCCCTTCCCCCBEEEEEEECTTSTTCSCTTC...,413,XRAY,1.7,0.19,0.22
4,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,108,XRAY,2.0,0.21,0.25


### Calculate Secondary Structure Content
To reduce the DSSP 8-state (q8) classification to a 3-state (q3) classification we group related classifications: alpha (I, H, G), beta (E, B), coil (S, T, C). Then we calculate the fraction of the amino acid residues in each of the three classes.

In [6]:
alpha_fraction = lambda s: (s.count('I') + s.count('H') + s.count('G')) / len(s)                                                   
beta_fraction = lambda s: (s.count('E') + s.count('B')) / len(s)  
coil_fraction = lambda s: (s.count('S') + s.count('T') + s.count('C')) / len(s)  
                                                                                                                                                             
# calculate each secondary structure fraction                                                                                                             
df['alpha'] = df.secondary_structure.apply(alpha_fraction)                                                                                         
df['beta'] = df.secondary_structure.apply(beta_fraction)                                                                                          
df['coil'] = df.secondary_structure.apply(coil_fraction)                                                                                           

df.head()

,pdbChainId,sequence,secondary_structure,length,Exptl.,resolution,R-factor,FreeRvalue,alpha,beta,coil
0,12AS.A,MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQD...,CCCCHHHHHHHHHHHHHHHHHHHHHHHCEEECCCCSEEETTSSCSC...,330,XRAY,2.2,0.16,0.29,0.345455,0.206061,0.448485
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,366,XRAY,2.1,0.19,0.26,0.469945,0.046448,0.483607
2,1A0I.A,VNIKTNPFKAVSFVESAIKKALDNAGYLIAEIKYDGVRGNICVDNT...,CTTCCCCEEEEECCHHHHHHHHHHHSSEEEEECCCSEEEEEEEETT...,348,XRAY,2.6,0.22,0.34,0.232759,0.318966,0.448276
3,1A12.C,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,CCCCCCCCCCCCCCCCCCCTTCCCCCBEEEEEEECTTSTTCSCTTC...,413,XRAY,1.7,0.19,0.22,0.038741,0.418886,0.542373
4,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,108,XRAY,2.0,0.21,0.25,0.037037,0.472222,0.490741


## Classify Structures by Secondary Structure Content
Next we classify each protein chain into one of four classes. We use a threshold of 25% to define a predominant class.

* alpha: predominantly alpha (>=25%)
* beta: predominantly beta (>=25%)
* alpha+beta: significant alpha (>=25%) and beta (>=25%)
* other: cases that do not fit into the 3 classes above 

Protein chains in the *other* class will be ignored in the subsequent analysis.

In [7]:
def protein_fold_type(data, minThreshold, maxThreshold):
    '''
    Returns fold type with three major secondary structure class:
    "alpha", "beta", "alpha+beta", and "other" based upon the fraction of alpha/beta content.

    Attributes:
        data: input dataframe with alpha, beta composition
        minThreshold (float): below this threshold, the secondary structure is ignored
        maxThreshold (float): above this threshold, the secondary structure is ignored
    '''
    if data.alpha > maxThreshold and data.beta < minThreshold:                                
        return "alpha"                                                                        
    elif data.beta > maxThreshold and data.alpha < minThreshold:                              
        return "beta"                                                                         
    elif data.alpha > maxThreshold and data.beta > maxThreshold:                              
        return "alpha+beta"                                                                   
    else:                                                                                     
        return "other"

In [8]:
df[value_col] = df.apply(protein_fold_type, minThreshold = 0.05, maxThreshold = 0.25, axis=1)

# exclude protein chains without a dominant classification from further analysis.
df = df[df[value_col] != 'other']

print("Dataset size", df.shape[0])
df.head(10)

Dataset size 5370


,pdbChainId,sequence,secondary_structure,length,Exptl.,resolution,R-factor,FreeRvalue,alpha,beta,coil,foldClass
1,16VP.A,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,366,XRAY,2.100,0.19,0.26,0.469945,0.046448,0.483607,alpha
3,1A12.C,RRSPPADAIPKSKKVKVSHRSHSTEPGLVLTLGQGDVGQLGLGENV...,CCCCCCCCCCCCCCCCCCCTTCCCCCBEEEEEEECTTSTTCSCTTC...,413,XRAY,1.700,0.19,0.22,0.038741,0.418886,0.542373,beta
4,1A1X.A,GSAGEDVGAPPDHLWVHQEGIYRDEYQRTWVAVVEEETSFLRARVQ...,CCCCCCCCCCCSEEEEEETTEEEETTSCEEEEEEEECSSCEEEEEE...,108,XRAY,2.000,0.21,0.25,0.037037,0.472222,0.490741,beta
5,1A2X.B,GDEEKRNRAITARRQHLKSVMLQIAATELEKEEGRREAEKQNYLAEH,CCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHTCCCCCCCCCCCCCCC,47,XRAY,2.300,0.22,0.33,0.574468,0.000000,0.425532,alpha
8,1A62.A,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...,CBHHHHHTSCHHHHHHHHHTTTCCCCTTSCHHHHHHHHHHHHHHTT...,130,XRAY,1.550,0.22,0.25,0.284615,0.261538,0.453846,alpha+beta
12,1A92.B,GREDILEQWVSGRKKLEELERDLRKLKKKIKKLEEDNPWLGNIKGI...,CHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCTTHHHHHHH...,50,XRAY,1.800,0.23,0.28,0.860000,0.000000,0.140000,alpha
14,1A9X.F,IKSALLVLEDGTQFHGRAIGATGSAVGEVVFNTSMTGYQEILTDPS...,CCEEEEEETTCCEEEEEECSCSEEEEEEEEEECCSSCHHHHHTCGG...,379,XRAY,1.800,0.19,1.00,0.250660,0.321900,0.427441,alpha+beta
16,1AEP.A,AAGHVNIAEAVQQLNHTIVNAAHELHETLGLPTPDEALNLLTEQAN...,CCCCCCHHHHHHHHHHHHHHHHHHHGGGGGSSCTTHHHHHHHHHHH...,161,XRAY,2.700,0.21,1.00,0.757764,0.000000,0.242236,alpha
17,1AH7.A,WSAEDKHKEGVNSHLWIVNRAIDIMSRNTTLVKQDRVAQLNEWRTE...,CCCCSSSCGGGCHHHHHHHHHHHHHHHCCSSCCHHHHHHHHHTHHH...,245,XRAY,1.501,0.20,0.23,0.632653,0.008163,0.359184,alpha
19,1AHS.A,TGPYAGAVEVQQSGRYYVPQGRTRGGYINSNIAEVCMDAGAAGQVN...,CCTTTTCSCBCCTTBCCCSSSSEEEEEEETTEEEEEECTTEEEECH...,126,XRAY,2.300,0.21,1.00,0.031746,0.444444,0.523810,beta


## Save Dataset
Here we save the representative dataset with protein sequence and fold classification as a Pandas dataframe for further analysis.

In [9]:
df.to_json("./secondaryStructure.json")

## Next step
After you saved the dataset here, go back to the [0-Workflow.ipynb](./0-Workflow.ipynb) to run the next step of the analysis.